# High-level Theano + Lasagne Example

In [1]:
import numpy as np
import os
import sys
import theano.tensor as T
import theano
import lasagne
import lasagne.layers as L
import lasagne.nonlinearities as nl
import lasagne.objectives as obj
import lasagne.updates as upd
from common.params import *
from common.utils import *

In [12]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Theano: ", theano.__version__)
print("Lasagne: ", lasagne.__version__)

OS:  linux
Python:  3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.12.1
Theano:  unknown
Lasagne:  0.2.dev1
0.01


In [3]:
def create_symbol():
    conv1 = L.Conv2DLayer(X, num_filters=50, filter_size=(3, 3), padding='same')
    conv2 = L.Conv2DLayer(conv1, num_filters=50, filter_size=(3, 3), padding='same')
    pool1 = L.MaxPool2DLayer(conv2, pool_size=(2, 2), strides=(2, 2))
    drop1 = L.DropoutLayer(pool1, 0.25)
    
    conv3 = L.Conv2DLayer(drop1, num_filters=100, filter_size=(3, 3), padding='same')
    conv4 = L.Conv2DLayer(conv3, num_filters=100, filter_size=(3, 3), padding='same')
    pool2 = L.MaxPool2DLayer(conv4, pool_size=(2, 2), strides=(2, 2))
    drop2 = L.DropoutLayer(pool2, 0.25)
    
    flatten = L.FlattenLayer(drop2)
    fc1 = L.DenseLayer(flatten, 512)
    drop4 = L.DropoutLayer(fc1, 0.5)
    pred = L.DenseLayer(drop4, N_CLASSES, name="output", nonlinearity=nl.softmax)
    
    return pred

In [4]:
def init_model(net):
    pred = L.get_output(net)
    params = L.get_all_params(net)
    xentropy = obj.categorical_crossentropy(pred, y)
    loss = T.mean(xentropy)
    # The tensorflow LR, MOMENTUM are slightly different
    updates = upd.momentum(loss, params, learning_rate=LR, momentum=MOMENTUM)
    return pred, loss, updates

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 3.86 s, sys: 1.5 s, total: 5.36 s
Wall time: 1min 42s


In [6]:
%%time
# Place-holders
X = L.InputLayer(shape=(None, 3, 32, 32))
y = T.ivector("y")
# Initialise model
net = create_symbol()

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 55.9 ms


In [7]:
%%time
pred, loss, updates = init_model(net)
# Accuracy for logging
accuracy = obj.categorical_accuracy(pred, y)
accuracy = T.mean(T.cast(accuracy, theano.config.floatX))

CPU times: user 816 ms, sys: 48 ms, total: 864 ms
Wall time: 965 ms


In [8]:
%%time
# Compile functions
train_func = theano.function([X.input_var, y], [loss, accuracy], updates=updates)
pred_func = theano.function([X.input_var], T.argmax(pred, axis=1))

CPU times: user 1.2 s, sys: 72 ms, total: 1.27 s
Wall time: 1.47 s


In [13]:
%%time
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        loss, acc_train = train_func(data, label)
    # Log
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.5625
1 Train accuracy: 0.765625


KeyboardInterrupt: 

In [16]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    output = pred_func(data)
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 1min 10s, sys: 920 ms, total: 1min 11s
Wall time: 18 s


In [17]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.641426282051
